In [235]:
import pypsa
import pandas as pd
import numpy as np
from pyomo.environ import Constraint
import logging


In [236]:
logging.basicConfig(level="INFO")

In [237]:
network = pypsa.Network()
network.set_snapshots(range(0, 24, 1))
n_buses=4
# Add the four buses to the network
network.add("Bus", "Bus 1", carrier="AC")
network.add("Bus", "Bus 2", carrier="Wind")
#network.add("Bus", "Bus 3", carrier="PV")
network.add("Bus", "Bus 4", carrier="Gas")


network.add("Carrier", "Wind")
#network.add("Carrier", "PV")
network.add("Carrier", "Gas", co2_emissions=0.2)



network.add("GlobalConstraint", "co2_limit", sense="<=", constant=0.0)


In [238]:
# Add the two generators to the network
#network.add("Generator", "Gas", bus="Bus 2", p_nom=800)
network.add("Generator", "gen", bus="Bus 1", capital_cost=300, p_nom=1200)


# Add the wind and solar generators to the network
network.add(
    "Generator", 
    "Wind", 
    bus="Bus 2", 
    carrier="Wind",
    p_nom_extendable=True,
    p_max_pu = 800 * np.array([0,0,0,0,0,0,0,0,0.5,1,1,1,1,1,1,1,0.7,0.5,0.3,0,0,0,0,0]),
    capital_cost=1000, 
    p_nom=800,    
)


In [239]:
# Add the two loads to the network
network.add("Load", "Load 1", bus="Bus 2", p_set=300)
#network.add("Load", "Load 2", bus="Bus 3", p_set=1000)


# Add the storage unit to the network

network.add("Bus", "Bus 3", carrier="Hydrogen")
network.add("Carrier", "Hydrogen")

network.add(
    "Link",
    "Electrolysis",
    bus0="Bus 2",
    bus1="Bus 3",
    efficiency=0.8,
    capital_cost=1000,
    p_nom_extendable=True,
)
network.add("Store", "Hydrogen", bus="Bus 3", e_cyclic=True, e_nom=1000)


In [240]:
network.add("Line", "Line 1", bus0="Bus 1", bus1="Bus 2",r=0.01, x=0.1, s_nom=1000)
network.add("Line", "Line 2", bus0="Bus 2", bus1="Bus 3",r=0.01, x=0.1, s_nom=1500)
#network.add("Line", "Line 3", bus0="Bus 1", bus1="Bus 3", x=0.1, s_nom=800)
network.add("Line", "Line 4", bus0="Bus 3", bus1="Bus 4",r=0.01, x=0.1, s_nom=1000)

In [241]:
#network.lopf()

In [242]:
#model = network.optimize.create_model()

In [243]:
network.optimize

In [244]:
network.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
Bus 1,1.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
Bus 2,1.0,,0.0,0.0,Wind,None,1.0,0.0,inf,PQ,
Bus 4,1.0,,0.0,0.0,Gas,None,1.0,0.0,inf,PQ,
Bus 3,1.0,,0.0,0.0,Hydrogen,None,1.0,0.0,inf,PQ,


In [245]:
network.loads

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
Load 1,Bus 2,,,300.0,0.0,-1.0


In [246]:
network.generators_t.p_max_pu

Generator,Wind
snapshot,
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,400.0


In [247]:
network.stores

attribute,bus,type,carrier,e_nom,e_nom_extendable,e_nom_min,e_nom_max,e_min_pu,e_max_pu,e_initial,...,e_cyclic_per_period,p_set,q_set,sign,marginal_cost,capital_cost,standing_loss,build_year,lifetime,e_nom_opt
Store,,,,,,,,,,,,,,,,,,,,,
Hydrogen,Bus 3,,,1000.0,False,0.0,inf,0.0,1.0,0.0,...,True,0.0,0.0,1.0,0.0,0.0,0.0,0,inf,0.0


In [250]:
network.links


attribute,bus0,bus1,type,carrier,efficiency,build_year,lifetime,p_nom,p_nom_extendable,p_nom_min,...,p_set,p_min_pu,p_max_pu,capital_cost,marginal_cost,length,terrain_factor,ramp_limit_up,ramp_limit_down,p_nom_opt
Link,,,,,,,,,,,,,,,,,,,,,
Electrolysis,Bus 2,Bus 3,,,0.8,0,inf,0.0,True,0.0,...,0.0,0.0,1.0,1000.0,0.0,0.0,1.0,NaN,NaN,0.0
